---

## BONUS: Schaalvergroting met PySpark

**Wanneer PySpark gebruiken?** Data > 10GB, 100+ files/dag, of processing > 30 min


**Key verschillen met pandas:**
- **Lazy evaluation**: PySpark bouwt een execution plan en voert pas uit bij `.collect()`, `.show()`, of `.write()`
- **Immutability**: DataFrames zijn immutable (elke transformatie = nieuwe DF)
- **Partitioning**: Data wordt automatisch verdeeld over cluster nodes
- **Fault tolerance**: Auto-recovery bij node failures


In [ ]:
# Als dit lokaal niet werkt, probeer het dan in Google Colab:
# https://colab.research.google.com/
!pip install pyspark==3.5.1

In [ ]:
# 1. Initialiseer Spark

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SensorPipeline") \
    .config("spark.sql.session.timeZone", "Europe/Amsterdam") \
    .getOrCreate()

print(f"Spark versie: {spark.version}")
print(f"Master: {spark.sparkContext.master}")


In [ ]:
# 2. Extract - lees alle JSON files parallel
# In plaats van Python loop, laat Spark dit parallel doen

from pyspark.sql.functions import to_timestamp, col

# Als JSON files een array bevatten ([]) - gebruik dan multiLine=True
sensor_df = spark.read.option("multiLine", "true").json("sample_data/sensor_readings_*.json")

print(f"Gelezen: {sensor_df.count()} records")
print("\nSchema:")
sensor_df.printSchema()

# Transformeer DIRECT na extract: timestamp van string naar timestamp type
sensor_df = sensor_df.withColumn("timestamp", to_timestamp(col("timestamp"), "yyyy-MM-dd'T'HH:mm:ss"))

sensor_df.show(3, truncate=False)

In [ ]:
# 3. Lees metadata
metadata_df = spark.read.csv("sample_data/sensor_metadata.csv", header=True, inferSchema=True)
metadata_df.show()

In [ ]:
# 4. Transform - declaratief en parallel

# Spark vs. Pandas:
# - Spark = lazy evaluation (bouwt execution plan, voert pas uit bij .show()/.write())
# - Pandas = eager evaluation (elke operatie wordt meteen uitgevoerd)
# - Spark's Catalyst optimizer kan deze chain optimaliseren voor beste performance

from pyspark.sql.functions import hour, dayofweek
from pyspark.sql.types import TimestampType

transformed_df = sensor_df \
    .withColumn("timestamp", col("timestamp").cast(TimestampType())) \
    .join(metadata_df.select("sensor_id", "sensor_type", "manufacturer", "model"),
          on="sensor_id", how="left") \
    .withColumn("hour_of_day", hour(col("timestamp"))) \
    .withColumn("day_of_week", dayofweek(col("timestamp"))) \
    .withColumn("is_working_hours",
                (col("hour_of_day").between(8, 18)) & (col("day_of_week").between(2, 6))) \
    .filter(
        ((col("sensor_type") == "TEMP") & col("value").between(10, 35)) |
        ((col("sensor_type") == "CO2") & col("value").between(300, 5000))
    )

transformed_df.show(5)

In [ ]:
# 5. Aggregaties - veel sneller dan pandas op grote datasets
from pyspark.sql.functions import date_format

hourly_avg = transformed_df \
    .groupBy("sensor_id", "sensor_type", date_format("timestamp", "yyyy-MM-dd HH:00:00").alias("hour")) \
    .agg({"value": "avg", "battery_level": "min"}) \
    .withColumnRenamed("avg(value)", "avg_value") \
    .withColumnRenamed("min(battery_level)", "min_battery") \
    .orderBy("hour", "sensor_id")

print("\nUurlijks gemiddelde per sensor:")
hourly_avg.show(10)

In [ ]:

# 6. Load - partitioned write (belangrijk voor query performance!)
# Partitioning = data fysiek gescheiden op disk per sensor_type
transformed_df.write \
    .mode("overwrite") \
    .partitionBy("sensor_type") \
    .parquet("warehouse_spark/")

print("\nData geschreven naar warehouse_spark/ (gepartitioneerd per sensor_type)")
print("Directory structuur:")
# ls -R warehouse_spark/ zou tonen:
# warehouse_spark/sensor_type=TEMP/part-00000.parquet
# warehouse_spark/sensor_type=CO2/part-00000.parquet

In [ ]:
# 7. Performance vergelijking (simulatie met dupes data)
# Maak 1000x copies voor simulatie
large_df = sensor_df
for i in range(10):  # 2^10 = 1024x data
    large_df = large_df.union(sensor_df)

print(f"\nSimulatie met {large_df.count()} records (1024x origineel)")

# Pandas zou hier out-of-memory gaan, PySpark blijft werken
# omdat het niet alles in geheugen hoeft te laden


In [ ]:
# 8. Spark UI - monitoring
print(f"\nSpark UI beschikbaar op: http://localhost:4040")
print("Hier zie je:")
print("  - Execution plans (DAG)")
print("  - Job stages en tasks")
print("  - Data shuffle operaties")
print("  - Node resource usage")

# Stop Spark
spark.stop()
